# Custom Chatbot Project

For this project, I chose a dataset comprised of four Wikipedia articles focused on various legal issues involving Donald Trump. 

These articles include details on his prosecutions in New York, indictments, the E. Jean Carroll lawsuit, and the Stormy Daniels scandal. This selection is appropriate for this task because the content of these articles was not included in the training data of the GPT-3.5 model used in this project, which has a knowledge cutoff in September 2021. 

By embedding and integrating these articles into our project, we can enable the language model to understand and generate information based on the latest and specific data that it originally would not have. This approach ensures that the model can provide accurate and up-to-date answers about these topics, despite their absence in its initial training dataset. 

Using these targeted embeddings allows us to bridge the knowledge gap effectively, demonstrating a practical application of embeddings in enhancing the model's performance on newer information.

## Prep

In [15]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [16]:

import pandas as pd
import requests
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI
from scipy.spatial.distance import cosine

# A file named ".env" is expected to be present in the same directory of this notebook, and it should have the OPENAI_API_KEY variable with the OpenAI token
load_dotenv()

COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
TOKENIZER_ENCODING = "cl100k_base"
MAX_TOKENS = 2048

QUESTION_1 = "Who is the judge presiding over the 'The People of the State of New York v. Donald J. Trump' trial?"
QUESTION_2 = "What are the indictments filed against Donald J. Trump?"

client = OpenAI()

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [17]:
wikipedia_articles = ["E._Jean_Carroll_v._Donald_J._Trump", "Stormy_Daniels–Donald_Trump_scandal",
                      "Prosecution_of_Donald_Trump_in_New_York", "Indictments_against_Donald_Trump"]

wikipedia_api_url_template = "https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exlimit=1&titles={article_name}&explaintext=1&formatversion=2&format=json"

wikipedia_urls = [wikipedia_api_url_template.format(article_name=article_name) for article_name in wikipedia_articles]

df = pd.DataFrame()
for wikipedia_url in wikipedia_urls:
    res = requests.get(wikipedia_url)
    article_df = pd.DataFrame()
    article_df["text"] = res.json()["query"]["pages"][0]["extract"].strip().split("\n")
    df = pd.concat([df, article_df], ignore_index=True)

df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]
df

,text
0,E. Jean Carroll v. Donald J. Trump is the name...
1,"In November 2022, Carroll filed her second sui..."
2,"In September 2023, Kaplan issued a partial sum..."
7,"On June 21, 2019, E. Jean Carroll published an..."
8,The allegations were made during the Trump adm...
...,...
512,Trump was indicted on state charges in a March...
517,Trump was indicted in June 2023 in the United ...
522,Trump was indicted in August 2023 in the Unite...
527,Trump was indicted on state charges in an Augu...


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [18]:
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = client.embeddings.create(
        input=df.iloc[i:i + batch_size]["text"].tolist(),
        model=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data.embedding for data in response.data])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,E. Jean Carroll v. Donald J. Trump is the name...,"[-0.014741996303200722, -0.0011683921329677105..."
1,"In November 2022, Carroll filed her second sui...","[-0.02475082129240036, -0.013566769659519196, ..."
2,"In September 2023, Kaplan issued a partial sum...","[-0.01716078259050846, 0.0019279149128124118, ..."
7,"On June 21, 2019, E. Jean Carroll published an...","[-0.025681884959340096, -0.00802887137979269, ..."
8,The allegations were made during the Trump adm...,"[-0.036411892622709274, 0.0017503190319985151,..."
...,...,...
512,Trump was indicted on state charges in a March...,"[-0.03097999095916748, -0.02709445357322693, -..."
517,Trump was indicted in June 2023 in the United ...,"[-0.022944077849388123, -0.029442088678479195,..."
522,Trump was indicted in August 2023 in the Unite...,"[-0.037209149450063705, -0.018463533371686935,..."
527,Trump was indicted on state charges in an Augu...,"[-0.021089570596814156, -0.026820292696356773,..."


In [19]:


def get_question_embeddings(question: str) -> list[float]:
    """
    Gets the embeddings of the given question using the OpenAI API.
    :param question: The question to answer.
    :return: The embeddings of the given question.
    """
    resp = client.embeddings.create(
        input=question,
        model=EMBEDDING_MODEL_NAME
    )
    return resp.data[0].embedding


def get_rows_sorted_by_relevance(question: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """

    # Get embeddings for the question text
    question_embeddings = get_question_embeddings(question)

    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    #
    # Modified based on https://github.com/openai/openai-python/blob/075ab0349a5a7d94528bfef0fd7adc8e2baf7c5d/openai/embeddings_utils.py#L139 
    # since embeddings_utils.py is no longer available in newer versions of the OpenAI SDK
    df_copy = df.copy()
    df_copy["distances"] = df_copy["embeddings"].apply(lambda x: cosine(question_embeddings, x))

    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant, so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy


def create_prompt(question: str, df: pd.DataFrame, max_token_count: int) -> str:
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding(TOKENIZER_ENCODING)

    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                          len(tokenizer.encode(question))

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


def answer_question(prompt: str, max_tokens: int = MAX_TOKENS) -> str:
    """
    Answers the question given a prompt using the OpenAI API.
    :param prompt: The question to answer.
    :param max_tokens: The maximum number of tokens to answer.
    :return: The answer.
    """
    try:
        completion = client.completions.create(
            prompt=prompt,
            model=COMPLETION_MODEL_NAME,
            max_tokens=max_tokens,
        )

        return completion.choices[0].text.strip()
    except Exception as e:
        print(e)
        return ""


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [20]:

first_prompt = f"""
Question: "{QUESTION_1}"
Answer:
"""
question_1_original_answer = answer_question(first_prompt)
question_1_embeddings_answer = answer_question(create_prompt(QUESTION_1, df, 1800), MAX_TOKENS)

print(f"""
Question: {QUESTION_1}
---------------------------------------------
Original Answer: {question_1_original_answer}
---------------------------------------------
Embeddings Answer: {question_1_embeddings_answer}
""")


Question: Who is the judge presiding over the 'The People of the State of New York v. Donald J. Trump' trial?
---------------------------------------------
Original Answer: As of March 2021, there is no active trial between the "The People of the State of New York" and Donald J. Trump. It is possible that you are referring to a past or future case. Please provide more information for an accurate answer.
---------------------------------------------
Embeddings Answer: Acting New York Supreme Court Justice Juan Merchan is presiding over the case.



### Question 2

In [21]:
second_prompt = f"""
Question: "{QUESTION_2}"
Answer:
"""
question_2_original_answer = answer_question(second_prompt, MAX_TOKENS)
question_2_embeddings_answer = answer_question(create_prompt(QUESTION_2, df, 1800), MAX_TOKENS)

print(f"""
Question: {QUESTION_2}
---------------------------------------------
Original Answer: {question_2_original_answer}
---------------------------------------------
Embeddings Answer: {question_2_embeddings_answer}
""")



Question: What are the indictments filed against Donald J. Trump?
---------------------------------------------
Original Answer: As of November 2021, there are no active indictments against former President Donald Trump. However, during his presidency, Trump's administration faced multiple investigations and legal challenges. These included:

- Special Counsel Robert Mueller's investigation into Russian interference in the 2016 election and possible collusion with the Trump campaign.
- The investigation by the Southern District of New York (SDNY) into potential campaign finance violations related to hush money payments made to women who allegedly had affairs with Trump.
- The New York Attorney General investigation into the Trump Organization's financial dealings and potential tax fraud.
- The defamation lawsuit filed by writer E. Jean Carroll, who accused Trump of sexually assaulting her in the mid-1990s.
- Various lawsuits filed by state attorneys general related to Trump's policies